# Burn-in Simulation

**Purpose**: Relax artificially injected particles with Jupiter at a fixed position (no migration).

This notebook uses the shared `simulation` module to avoid code duplication.

## Workflow
1. Configure simulation parameters
2. Run simulation (or load from checkpoint)
3. Analyze and visualize results

## 1. Setup and Imports

In [ ]:
# Add simulation module to path
import sys
sys.path.insert(0, '.')

from simulation import (
    SimulationConfig,
    run_simulation,
    load_checkpoint,
    plot_trojan_evolution,
    plot_snapshot,
    create_animation,
    compute_population_summary
)

print('Simulation module loaded successfully!')

## 2. Configuration

Adjust these parameters for your burn-in simulation:

In [ ]:
# === BURN-IN CONFIGURATION ===

# Duration in thousands of years
DURATION_KYR = 50

# Jupiter's semi-major axis (AU)
JUPITER_A = 5.2

# Number of planetesimals
N_PLANETESIMALS = 10000

# Input file (set to None to start fresh, or path to continue from previous)
INPUT_FILE = None  # e.g., 'jupiter_burnin_5kyr_5.2au.pkl'

# Number of snapshots to save
N_SNAPSHOTS = 200

# Create configuration
config = SimulationConfig.burnin(
    duration_kyr=DURATION_KYR,
    jupiter_a=JUPITER_A,
    input_file=INPUT_FILE,
    n_planetesimals=N_PLANETESIMALS,
    n_snapshots=N_SNAPSHOTS
)

print(f'Configuration: {config.name}')
print(f'  Duration: {config.end_time:.0f} years')
print(f'  Jupiter at: {config.jupiter_initial_a} AU')
print(f'  Planetesimals: {config.n_planetesimals}')
print(f'  Output: {config.output_file}')

## 3. Run Simulation

This will take some time depending on the duration.

In [ ]:
# Run the simulation
results = run_simulation(config, verbose=True)

## 4. Analysis and Visualization

In [ ]:
# If loading from saved file instead of running:
# results = load_checkpoint(config.output_file)

times = results['times']
l4_counts = results['l4_counts']
l5_counts = results['l5_counts']
snapshots_massive = results['snapshots_massive']
snapshots_planetesimals = results['snapshots_planetesimals']

print(f'Loaded {len(times)} snapshots')
print(f'Time range: {times[0]:.1f} - {times[-1]:.1f} yr')

In [ ]:
# Plot Trojan population evolution
fig = plot_trojan_evolution(
    times, l4_counts, l5_counts,
    title=f'Trojan Evolution ({config.name})'
)
fig.savefig(f'{config.name}_evolution.png', dpi=150)
print(f'Saved: {config.name}_evolution.png')

In [ ]:
# Population summary
summary = compute_population_summary(times, l4_counts, l5_counts)

print('=' * 60)
print('POPULATION SUMMARY')
print('=' * 60)
print(f"Initial: L4={summary['initial_l4']}, L5={summary['initial_l5']}, Total={summary['initial_total']}")
print(f"Final:   L4={summary['final_l4']}, L5={summary['final_l5']}, Total={summary['final_total']}")
print(f"Initial L4/L5 ratio: {summary['initial_ratio']:.3f}")
print(f"Final L4/L5 ratio:   {summary['final_ratio']:.3f}")
print(f"L4/L5 ratio slope:   {summary['ratio_slope']:.3e} yr⁻¹")
print(f"Retention: {summary['retention_percent']:.1f}%")
print('=' * 60)

In [ ]:
# Plot final snapshot
fig = plot_snapshot(
    snapshots_massive[-1],
    snapshots_planetesimals[-1],
    times[-1],
    show_trojan_zones=True
)
fig.savefig(f'{config.name}_final.png', dpi=150)
print(f'Saved: {config.name}_final.png')

## 5. Create Animation (Optional)

In [ ]:
# Create animated GIF (takes a minute)
gif_file = create_animation(
    times,
    snapshots_massive,
    snapshots_planetesimals,
    output_file=f'{config.name}.gif',
    fps=15,
    dpi=80
)

# Display the animation
from IPython.display import Image
Image(filename=gif_file)